In [ ]:
import pandas as pd

df = pd.read_csv("Jumlah_Kasus_HIV_AIDS_Jayapura_2023_clean.csv")

df.head(5) 


,Wilayah,HIV,AIDS,Jumlah,Mati
0,Kota Jayapura,3206,4747,7953,375
1,Provinsi Papua,21947,30846,52793,3735


In [ ]:
print(df.info())
print(df.describe())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2 entries, 0 to 1
Data columns (total 5 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   Wilayah  2 non-null      object
 1   HIV      2 non-null      int64 
 2   AIDS     2 non-null      int64 
 3   Jumlah   2 non-null      int64 
 4   Mati     2 non-null      int64 
dtypes: int64(4), object(1)
memory usage: 212.0+ bytes
None
                HIV          AIDS        Jumlah         Mati
count      2.000000      2.000000      2.000000     2.000000
mean   12576.500000  17796.500000  30373.000000  2055.000000
std    13251.888186  18454.779882  31706.668068  2375.878785
min     3206.000000   4747.000000   7953.000000   375.000000
25%     7891.250000  11271.750000  19163.000000  1215.000000
50%    12576.500000  17796.500000  30373.000000  2055.000000
75%    17261.750000  24321.250000  41583.000000  2895.000000
max    21947.000000  30846.000000  52793.000000  3735.000000
